<a href="https://colab.research.google.com/github/zeeez225/Tasks/blob/main/%D0%A3%D0%B3%D1%80%D0%BE%D0%B7%D1%8B_%D0%BD%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3  # Работа с базами данных SQLite
import subprocess  # Для выполнения системных команд
import json  # Работа с JSON (безопасная сериализация/десериализация)
import pickle  # Опасная библиотека для сериализации объектов Python
import matplotlib.pyplot as plt  # Библиотека для построения графиков

In [2]:
# --- Угроза 1: SQL Injection ---

def unsafe_sql_query(user_input):  # Функция демонстрирует SQL-инъекцию
    conn = sqlite3.connect(':memory:')  # Создание временной БД в оперативной памяти
    c = conn.cursor()  # Получение курсора для выполнения SQL-запросов
    c.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, username TEXT)")  # Создание таблицы users
    c.execute("INSERT INTO users (username) VALUES ('admin')")  # Вставка тестовой записи
    query = f"SELECT * FROM users WHERE username = '{user_input}'"  # Уязвимый SQL-запрос — строка собирается вручную
    print(f"Выполняется запрос: {query}")  # Логирование запроса (может быть полезно, но опасно)
    try:
        c.execute(query)  # Выполнение SQL-запроса
        result = c.fetchall()  # Получение результатов
        return result  # Возврат результата
    except sqlite3.Error as e:  # Обработка ошибок SQL
        return f"Ошибка SQL: {e}"  # Возврат сообщения об ошибке
    finally:
        conn.close()  # Закрытие соединения с БД

def safe_sql_query(user_input):  # Безопасная альтернатива
    conn = sqlite3.connect(':memory:')  # Временная БД в памяти
    c = conn.cursor()  # Получение курсора
    c.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, username TEXT)")  # Создание таблицы
    c.execute("INSERT INTO users (username) VALUES ('admin')")  # Вставка записи
    query = "SELECT * FROM users WHERE username = ?"  # Безопасный параметризованный запрос
    print(f"Выполняется безопасный запрос с параметром: {user_input}")  # Лог запроса
    c.execute(query, (user_input,))  # Выполнение безопасного запроса с параметром
    result = c.fetchall()  # Получение результата
    conn.close()  # Закрытие соединения
    return result  # Возврат результата


In [3]:
# --- Угроза 2: Командные инъекции ---

def unsafe_os_command(user_input):  # Пример уязвимости командной инъекции
    cmd = f"echo {user_input}"  # Сборка команды из строки (уязвимо)
    print(f"Выполняется команда: {cmd} (опасно)")  # Лог команды
    # subprocess.Popen(cmd, shell=True)  # Опасно: shell=True позволяет инъекции
    return cmd  # Возврат строки команды

def safe_os_command(user_input):  # Безопасная реализация
    import shlex  # Модуль для безопасного экранирования аргументов
    safe_input = shlex.quote(user_input)  # Экранирование пользовательского ввода
    cmd = ["echo", user_input]  # Список аргументов — безопаснее, чем строка
    print(f"Выполняется безопасная команда: {cmd}")  # Лог безопасной команды
    # subprocess.Popen(cmd)  # Безопасное выполнение
    return cmd  # Возврат команды

In [4]:
# --- Угроза 3: Утечка данных ---

def unsafe_logging(password):  # Уязвимость: логирование чувствительных данных
    print(f"Логируем пароль: {password}")  # Плохая практика: пароль виден в логах

def safe_logging(password):  # Безопасное логирование
    import hashlib  # Импорт библиотеки для хэширования
    hashed = hashlib.sha256(password.encode()).hexdigest()  # Хэширование пароля
    print(f"Логируем хэш пароля: {hashed}")  # Логирование безопасного хэша

In [5]:
# --- Угроза 4: Опасная десериализация pickle ---

def unsafe_deserialize(data):  # Пример уязвимости pickle
    try:
        obj = pickle.loads(data)  # Десериализация данных — может быть опасна
        print("Десериализация выполнена")  # Успешная десериализация
        return obj  # Возврат объекта
    except Exception as e:  # Обработка ошибок
        print(f"Ошибка десериализации: {e}")  # Сообщение об ошибке
def safe_deserialize(data):  # Безопасная альтернатива через JSON
    try:
        obj = json.loads(data)  # Десериализация JSON (безопаснее)
        print("Безопасная десериализация JSON выполнена")  # Успех
        return obj  # Возврат объекта
    except Exception as e:  # Обработка ошибок
        print(f"Ошибка десериализации JSON: {e}")  # Ошибка при десериализации

In [6]:
# --- Визуализация распространенности угроз ---

def plot_threats():  # Функция для отображения статистики угроз
    threats = ['SQL Injection', 'Командные инъекции', 'Утечка данных', 'Опасная десериализация', 'Устаревшие библиотеки']  # Список угроз
    counts = [45, 25, 15, 10, 5]  # Примерные данные по угрозам (в процентах)
    plt.figure(figsize=(8,5))  # Размер графика
    bars = plt.bar(threats, counts, color='crimson')  # Создание столбчатой диаграммы
    plt.title('Распределение основных угроз безопасности Python-приложений (%)')  # Заголовок графика
    plt.ylabel('Процент случаев')  # Подпись оси Y
    plt.ylim(0, 50)  # Ограничение шкалы Y
    for bar in bars:  # Добавление надписей над столбцами
        yval = bar.get_height()  # Получение высоты столбца
        plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f'{yval}%', ha='center', va='bottom')  # Отображение значения
    plt.show()  # Отображение графика

In [8]:
# --- Примеры вызова и выводы ---

if name == "main":  # Точка входа в программу
    print("=== Пример SQL Injection ===")  # Заголовок
    malicious_input = "' OR '1'='1"  # Вредоносный ввод
    print("Результат небезопасного запроса:", unsafe_sql_query(malicious_input))  # Демонстрация уязвимости
    print("Результат безопасного запроса:", safe_sql_query(malicious_input))  # Демонстрация защиты

    print("\n=== Пример командной инъекции ===")  # Заголовок
    dangerous_input = "hello; rm -rf /"  # Потенциально опасный ввод
    print("Небезопасная команда:", unsafe_os_command(dangerous_input))  # Уязвимая реализация
    print("Безопасная команда:", safe_os_command(dangerous_input))  # Защищённая реализация

    print("\n=== Пример утечки данных ===")  # Заголовок
    unsafe_logging("SuperSecret123")  # Логирование пароля (опасно)
    safe_logging("SuperSecret123")  # Безопасное логирование

    print("\n=== Пример десериализации ===")  # Заголовок
    malicious_pickle = pickle.dumps({"key": "value"})  # Создание объекта для pickle
    unsafe_deserialize(malicious_pickle)  # Опасная десериализация
    safe_deserialize('{"key": "value"}')  # Безопасная десериализация через JSON

    print("\n=== Визуализация угроз ===")  # Заголовок
    plot_threats()  # Построение графика

    print("\n=== Выводы ===")  # Заключение
    print("""
    1. SQL Injection можно предотвратить с помощью параметризации запросов.
    2. Командные инъекции устраняются отказом от shell=True и экранированием ввода.
    3. Никогда не логируйте пароли или чувствительные данные в открытом виде.
    4. Не используйте pickle для десериализации данных из ненадежных источников — предпочтите JSON.
    5. Регулярно обновляйте Python и библиотеки, чтобы избежать уязвимостей.
        """)

NameError: name 'name' is not defined